In [308]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Random

In [309]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("sparseblocks.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl")

reference_trajectory

## Reference Trajectory

## Problem Definition

In [310]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Discretization
tf = 0.6
dt = 0.01
N = Int(ceil(tf / dt)) + 1
times = range(0, tf, length=N)
t_trans = 0.25
n = 14
m = 4

# Initial Conditions
# currently, we assume the initial mode ID is 1
xinit = zeros(n)
xinit[1]  = -lb / 2.5                  # xb
xinit[2]  = sqrt(l1^2 + l2^2) + 0.05 # yb
xinit[3]  = -30 * pi / 180            # theta
xinit[6]  = -lb                      # x2
xinit[7]  = 0.25                      # y2
xinit[9]  = -1.0                     # yb_dot
xinit[12] = -1.0                     # y2_dot

xterm = zeros(n)
xterm[1] = -lb / 2           # xb
xterm[2] = sqrt(l1^2 + l2^2) # yb
xterm[6] = -lb               # x2

init_mode = 1

# Reference Trajectory
Xref, Uref = reference_trajectory(model, times, t_trans, xinit, xterm, init_mode)

# Objective
Random.seed!(1)
Q = Diagonal([10.0;10.0;1.0; 10.0;10.0; 10.0;10.0; 10.0;10.0;1.0; 10.0;10.0; 10.0;10.0])
R = Diagonal(fill(1e-3, 4))
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q, R, Xref[k], Uref[k])
end
push!(obj, LQRCost(Qf, R * 0, Xref[N], Uref[1]))

# Define the NLP
# function HybridNLP(model, obj, init_mode, tf, N, x0, xf)

# nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end]);
nlp = HybridNLP(model, obj, init_mode, tf, N, xinit, xterm);

## Solve

In [311]:
nlp.cinds

6-element Vector{UnitRange{Int64}}:
 1:14
 15:28
 29:868
 869:929
 930:970
 971:1031

In [312]:
# Initial guess
xtransit = zeros(n)
xtransit[1] = -lb/2 # xb
xtransit[2] = l1    # yb
xtransit[3] = 0     # theta
xtransit[6] = -lb   # x2

# Random.seed!(1)
# Uguess = [u + 0.1*randn(length(u)) for u in Uref]
# Xguess = [x + 0.1*randn(length(x)) for x in Xref]

Uguess = [zeros(m) + rand(m) for u in Uref]
Xguess = [zeros(n) for x in Xref]

k_trans = nlp.k_trans

for i = 1:k_trans
    Xguess[i] = xinit + (xtransit - xinit)/k_trans * i + 0.1 * rand(n)
end

# display((xtransit - xinit)/timesteps_phase_1)

In [313]:
nlp.xf

14-element MVector{14, Float64} with indices SOneTo(14):
 -0.25
  0.3535533905932738
  0.0
  0.0
  0.0
 -0.5
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [314]:
Z0 = packZ(nlp, Xguess, Uguess);

# Z_sol, solver = solve(Z0, nlp, c_tol=1e-6, tol=1e-6)
Z_sol, solver = solve(Z0, nlp, c_tol=1e-4, tol=1e-3)

([-0.19999999995346743, 0.4035533906082208, 0.5235987755975288, -2.0094423339880293e-11, 5.406876755236595e-13, -0.4999999999997654, 0.2500000000002547, 7.824894521512318e-12, -0.9999999999973532, -3.540761817651308e-13  …  1.3029009672981952e-14, -0.5000000000137698, 1.3029006544687907e-14, 3.0157601118488365e-12, 2.0990192710383805e-12, -1.234816430200708e-13, 4.985116525161758e-13, 3.4534421805299843e-15, 2.5934181351424882e-12, 3.4534825137681947e-15], Ipopt.Optimizer)

In [315]:
Z_sol[1:14] - xinit

14-element Vector{Float64}:
  4.6532583342084877e-11
  1.4947043602830945e-11
 -7.700506898800086e-13
 -2.0094423339880293e-11
  5.406876755236595e-13
  2.345901251032956e-13
  2.546851618490109e-13
  7.824894521512318e-12
  2.646771690706373e-12
 -3.540761817651308e-13
 -9.728207544363355e-13
  3.5560443478743764e-13
  5.692484607924333e-14
  7.092940776471933e-14

In [316]:
Z_sol[end-13:end] - xterm

14-element Vector{Float64}:
 -1.1950440637065185e-11
 -1.0382139592479689e-11
  7.499257777116727e-13
 -9.440180763563116e-13
  1.3029009672981952e-14
 -1.3769763107518429e-11
  1.3029006544687907e-14
  3.0157601118488365e-12
  2.0990192710383805e-12
 -1.234816430200708e-13
  4.985116525161758e-13
  3.4534421805299843e-15
  2.5934181351424882e-12
  3.4534825137681947e-15

In [317]:
y2 = zeros(N)
for k = 1:N
    y2[k] = Z_sol[7 + 18*(k-1)]
end
@show y2

61-element Vector{Float64}:
 0.2500000000002547
 0.24960891973170918
 0.24885095571694013
 0.24814220773209295
 0.24748074739521664
 0.24686231939470063
 0.24628405410287812
 0.24574389202593577
 0.24524042206229452
 0.24477244565234121
 ⋮
 4.565065687211366e-14
 5.306437235715603e-14
 6.077955859100761e-14
 6.80032762328289e-14
 7.323123733709754e-14
 7.340581147374008e-14
 6.351440541289785e-14
 3.9182303656394e-14
 1.3029006544687907e-14

In [318]:
# display(Z_sol)
using DelimitedFiles
using CSV
writedlm( "data.csv",  Z_sol, ',')